<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# NLmod examples  

This example notebook shows a basic example of a model created with the `nlmod` package. The `nlmod` package contains functions to create modflow models anywhere in the Netherlands.

### Contents<a name="TOC"></a>
0. [Download MODFLOW-binaries](#binaries)
1. [Create model](#create)
2. [Run model](#run)
3. [Visualise](#visualise)

In [ ]:
import logging
import os

import flopy
import geopandas as gpd
import matplotlib.pyplot as plt
import nlmod


In [ ]:
print(f"nlmod version: {nlmod.__version__}")

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)


### [0. Download MODFLOW-binaries](#TOC)<a name="binaries"></a>
To run MODFLOW, we need to download the MODFLOW-excecutables. We do this with the following code:

In [ ]:
if not nlmod.util.check_presence_mfbinaries():
    nlmod.util.download_mfbinaries()


### [1. Create model](#TOC)<a name="create"></a>

With the code below we create a modflow model with the name 'IJmuiden'. This model has the following properties :
- an extent that covers part of the Northsea, Noordzeekanaal and the small port city IJmuiden.
- a structured grid based on the subsurface models [Regis](https://www.dinoloket.nl/regis-ii-het-hydrogeologische-model) and [Geotop](https://www.dinoloket.nl/detaillering-van-de-bovenste-lagen-met-geotop). The Regis layers that are not present within the extent are removed. In this case we use 'MSz1' as the bottom layer of the model. Use `nlmod.read.regis.get_layer_names()` to get all the layer names of Regis. All Regis layers below this layer are not used in the model. Geotop is used to replace the holoceen layer in Regis because there is no kh or kv defined for the holoceen in Regis. Part of the model is in the North sea. Regis and Geotop have no data there. Therefore the Regis and Geotop layers are extrapolated from the shore and the seabed is added using bathymetry data from [Jarkus](https://www.openearth.nl/rws-bathymetry/2018.html).
- starting heads of 1 in every cell.
- the model is a steady state model of a single time step.
- big surface water bodies (Northsea, IJsselmeer, Markermeer, Noordzeekanaal) within the extent are added as a general head boundary. The surface water bodies are obtained from a [shapefile](..\data\opp_water.shp).
- surface drainage is added using [ahn](https://www.ahn.nl) data and a default conductance of $1000 m^2/d$
- recharge is added using data from the [knmi](https://www.knmi.nl/nederland-nu/klimatologie/daggegevens) using the following steps:~~
    1. Check for each cell which KNMI weather and/or rainfall station is closest.
    2. Download the data for the stations found in 1. for the model period. For a steady state stress period the average precipitation and evaporation of 8 years before the stress period time is used.
    3. Combine precipitation and evaporation data from step 2 to create a recharge time series for each cell
    4. Add the timeseries to the model dataset and create the recharge package.
- constant head boundaries are added to the model edges in every layer. The starting head is used as constant head.

In [ ]:
# model settings
model_ws = "model1"
model_name = "IJmuiden"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
extent = [95000.0, 105000.0, 494000.0, 500000.0]
delr = 100.0
delc = 100.0
steady_state = True
start_time = "2015-1-1"
use_regis = True
regis_botm_layer = "MSz1"
use_geotop = True
add_northsea = True
starting_head = 1.0


In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent,
    use_regis=use_regis,
    regis_botm_layer=regis_botm_layer,
    use_geotop=use_geotop,
    cachedir=cachedir,
    cachename="combined_layer_ds.nc",
)

# create a model ds by changing grid of layer_model
ds = nlmod.mdims.to_model_ds(
    layer_model, model_name, model_ws, delr=delr, delc=delc
)

# add time discretisation
ds = nlmod.mdims.set_ds_time(
    ds, start_time=start_time, steady_state=steady_state, perlen=365 * 5
)

if add_northsea:
    ds = nlmod.mdims.add_northsea(ds)


In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# create ims
ims = nlmod.gwf.ims(sim)

# Create discretization
dis = nlmod.gwf.dis(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=starting_head)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)


In [ ]:
# voeg grote oppervlaktewaterlichamen toe o.b.v. rws shape
da_name = "rws_oppwater"
rws_ds = nlmod.read.rws.get_surface_water(
    ds, da_name, cachedir=ds.cachedir, cachename=da_name
)
ds.update(rws_ds)
ghb = nlmod.gwf.ghb(ds, gwf, da_name)


In [ ]:
# surface level drain
ahn_ds = nlmod.read.ahn.get_ahn(ds, cachedir=ds.cachedir, cachename="ahn")
ds.update(ahn_ds)

drn = nlmod.gwf.surface_drain_from_ds(ds, gwf)


# add constant head cells at model boundaries
ds.update(nlmod.gwf.constant_head.chd_at_model_edge(ds, ds["idomain"]))
chd = nlmod.gwf.chd(ds, gwf, head="starting_head")


In [ ]:
# add knmi recharge to the model datasets
knmi_ds = nlmod.read.knmi.get_recharge(
    ds, cachedir=ds.cachedir, cachename="recharge"
)
ds.update(knmi_ds)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)


A big part of the model data is stored in the variable `ds` which is an `xarray.Dataset`. The data is shown below.

In [ ]:
ds

### [2. Write and Run](#TOC)<a name="run"></a>
Now that we've created all the modflow packages we need to write them to modflow files. You always have to write the modflow data to the model workspace before you can run the model. You can write the model files and run the model using the function `nlmod.sim.write_and_run)` as shown below. This function has two additional options:
1. Write the model dataset to the disk if `write_ds` is `True`. This makes it easier and faster to load model data if you ever need it. 
2. Write a copy of this Jupyter Notebook to the same directory as the modflow files if `nb_path` is the name of this Jupyter Notebook. It can be useful to have a copy of the script that created the modflow files, together with the files. 

In [ ]:
nlmod.sim.write_and_run(
    sim, ds, write_ds=True, nb_path="01_basic_model.ipynb"
)


### [3. Visualise](#TOC)<a name="visualise"></a>

Using the `ds` and `gwf` variables it is quite easy to visualise model data. Below the modelgrid together with the surface water is shown.

In [ ]:
ax = nlmod.visualise.plots.plot_modelgrid(ds, gwf)
ax.figure.savefig(
    os.path.join(ds.figdir, "mgrid_swater.png"), bbox_inches="tight"
)


Data from a model with a structured grid can be easily visualised using the model dataset. Below some examples

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 11))
ds["ahn"].plot(ax=axes[0][0])
ds["botm"][0].plot(ax=axes[0][1])
ds["idomain"][0].plot(ax=axes[1][0])
ds["chd"][0].plot(ax=axes[1][1])
for axes1 in axes:
    for ax in axes1:
        ax.axis("scaled")

fig.savefig(
    os.path.join(ds.figdir, "ahn_bot_idom_chd.png"), bbox_inches="tight"
)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 11))
ds["bathymetry"].plot(ax=axes[0][0])
ds["northsea"].plot(ax=axes[0][1])
ds["kh"][1].plot(ax=axes[1][0])
ds["recharge"].plot(ax=axes[1][1])

for axes1 in axes:
    for ax in axes1:
        ax.axis("scaled")
fig.savefig(
    os.path.join(ds.figdir, "bath_nsea_kh_top.png"), bbox_inches="tight"
)
